In [1]:
import malaya

## Use Sastrawi stemmer

In [2]:
malaya.sastrawi_stemmer('saya tengah berjalan')

'saya tengah jalan'

In [3]:
malaya.sastrawi_stemmer('saya tengah berjalankan sangat-sangat')

'saya tengah jalan sangat'

In [4]:
malaya.sastrawi_stemmer('menarik')

'tarik'

## Load deep learning model

I really not suggest you to use this model. Use Sastrawi instead. We are adding our own rules into Sastrawi stemmer

In [5]:
stemmer = malaya.deep_stemmer()

downloading JSON stemmer


1.00MB [00:00, 260MB/s]                    
  0%|          | 0.00/21.4 [00:00<?, ?MB/s]

downloading stemmer graph


22.0MB [00:06, 3.92MB/s]                          


In [6]:
stemmer.stem('saya tengah berjalankan sangat-sangat')

'saya tengah jalan sangat sangat'

In [7]:
stemmer.stem('saya sangat sukakan awak')

'saya sangat suka awak'

In [8]:
stemmer.stem('saya sangat suakkan awak')

'saya sangat suak awak'